# Evaluating the GPT models on the SPOT Dataset
# Currently using the Chat.OpenAI.GPT3.5/4  Engine

# Adding Contextual data explaining the Indicator Terms

In [1]:
# Use the experiment output from one of the EleutherAI GPT-Neo models to generate the list of questions to pass into the Chat.OpenAI interface
# We use the version of Eleuther that processes Yes/No questions with zero-shot context
import pandas as pd
import ujson as json
import pprint
import os
import yaml
import openai
openai.api_key = open('./openai.key.temp','r').read()

from util_openai import OpenAICompletion

pp = pprint.PrettyPrinter(indent=4, width=260)
path_preds = '../output/spot/EleutherAI_gpt-neox-20b/_PSyn_AGmv_POrb_DId_RPH/predictions.csv'
experiment_output = pd.read_csv(path_preds)

In [2]:
# Preprocessing Questions to Pass to Chat.OpenAI
preds_prompts = experiment_output['preds_prompts'].tolist()
preds_prompts = [json.decode(x) for x in preds_prompts]

# Remove the text 'Question:' from the start of each prompt
preds_prompts = [ x[0].replace('Question: ', '') for x in preds_prompts]

# Remove double qoutation marks in each prompt
preds_prompts = [ x.replace('""', '"') for x in preds_prompts]

# Remove the text 'Answer:' from the start of each prompt
preds_prompts_direct = [x.replace('\nAnswer:', '') for x in preds_prompts]
preds_prompts_indirect = [x.replace('directly', 'indirectly') for x in preds_prompts_direct]
preds_prompts_normal = [x.replace('directly ', '') for x in preds_prompts_direct]


In [3]:
oac = OpenAICompletion(
    openai_model='gpt-3.5-turbo',
    max_tokens=5,
    temperature=0.75,
    system_start='Assistant is an intelligent chatbot designed to help users answers questions about whether government spending on "budget item" affects a "socio-economic indicator".',
    )

In [4]:
responses_direct = oac.get_responses(preds_prompts_direct)
responses_indirect = oac.get_responses(preds_prompts_indirect)
responses_normal = oac.get_responses(preds_prompts_normal)

 73%|███████▎  | 299/412 [15:25<05:26,  2.89s/it]

HTTP code 502 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text

 42%|████▏     | 172/412 [10:17<13:34,  3.39s/it]  

HTTP code 502 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text

100%|██████████| 412/412 [30:45<00:00,  4.48s/it]   


In [5]:
# Converting Responses to Yes/No/Nan
def convert_to_yes_no_nan(response):
    if 'no' in response.lower():
        return 'No'
    if 'yes' in response.lower():    
        return 'Yes'
    else:
        return 'NA'

responses_direct = [convert_to_yes_no_nan(x) for x in responses_direct]
responses_indirect = [convert_to_yes_no_nan(x) for x in responses_indirect]
responses_normal = [convert_to_yes_no_nan(x) for x in responses_normal]

In [6]:
import copy
# Save Predictions to File
# Drop the following columns from experiment_output, preds_ensemble_parsed, preds_ensemble, preds_prompts
experiment_output = experiment_output.drop(columns=['preds_ensemble_parsed', 'preds_ensemble', 'preds_prompts','preds_aggregate'], axis=1, errors='ignore')

# Setting up the experiment_output dataframe to save the predictions
experiment_output_direct = copy.deepcopy(experiment_output)
experiment_output_indirect = copy.deepcopy(experiment_output)
experiment_output_normal = copy.deepcopy(experiment_output)

# Add the predictions to the experiment_output dataframes
experiment_output_direct['preds_aggregate'] = responses_direct
experiment_output_direct['preds_prompts'] = [json.encode(x) for x in preds_prompts_direct]

experiment_output_indirect['preds_aggregate'] = responses_indirect
experiment_output_indirect['preds_prompts'] = [json.encode(x) for x in preds_prompts_indirect]

experiment_output_normal['preds_aggregate'] = responses_normal
experiment_output_normal['preds_prompts'] = [json.encode(x) for x in preds_prompts_normal]

# Create the config files
config_direct = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'directly', 'remove_public_health':True}
config_indirect = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'indirectly', 'remove_public_health':True}
config_normal = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'neutral', 'remove_public_health':True}

In [7]:
#Creating file directories
dir_ = '../output'

dir_direct = 'spot/gpt35turbo/directly_RPH'
dir_indirect = 'spot/gpt35turbo/indirectly_RPH'
dir_normal = 'spot/gpt35turbo/neutral_RPH'

os.makedirs(os.path.join(dir_, dir_direct), exist_ok=True)
os.makedirs(os.path.join(dir_, dir_indirect), exist_ok=True)
os.makedirs(os.path.join(dir_, dir_normal), exist_ok=True)

# Saving the experiment_output dataframes 
experiment_output_direct.to_csv(os.path.join(dir_, dir_direct, 'predictions.csv'),index=False)
experiment_output_indirect.to_csv(os.path.join(dir_, dir_indirect, 'predictions.csv'),index=False)
experiment_output_normal.to_csv(os.path.join(dir_, dir_normal, 'predictions.csv'),index=False)

# Saving the config files
with open(os.path.join(dir_, dir_direct, 'config.yaml'), 'w') as f:
    yaml.dump(config_direct, f)
    
with open(os.path.join(dir_, dir_indirect, 'config.yaml'), 'w') as f:
    yaml.dump(config_indirect, f)
    
with open(os.path.join(dir_, dir_normal, 'config.yaml'), 'w') as f:
    yaml.dump(config_normal, f)
